<a href="https://colab.research.google.com/github/ismailyou/ner/blob/main/Tebyan_Arabic_NER_Model_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install simpletransformers==0.61.5 nltk==3.5

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
import logging
import re

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

## Load the Model

In [ ]:
# Load the Model
custom_labels = ["O", "B-job", "I-job", "B-nationality", "B-person", "I-person", "B-location",
                 "B-time", "I-time", "B-event", "I-event", "B-organization", "I-organization",
                 "I-location", "I-nationality", "B-product", "I-product", "B-artwork", "I-artwork"]

model_args = NERArgs()
model_args.labels_list=custom_labels
model_args.silent = True

ner_model = NERModel(
     "xlmroberta", "marefa-nlp/marefa-ner",
     args=model_args,
     use_cuda=True # set to False to use CPU
)

## Model Inference

In [ ]:
# Model Inference
samples = [
    "تلقى تعليمه في الكتاب ثم انضم الى الأزهر عام 1873م. تعلم على يد السيد جمال الدين الأفغاني والشيخ محمد عبده",
    "بعد عودته إلى القاهرة، التحق نجيب الريحاني فرقة جورج أبيض، الذي كان قد ضمَّ - قُبيل ذلك - فرقته إلى فرقة سلامة حجازي . و منها ذاع صيته",
    "امبارح اتفرجت على مباراة مانشستر يونايتد مع ريال مدريد في غياب الدون كرستيانو رونالدو",
    "Government extends flight ban from India, Pakistan until June 21"
]

# Preprocess
samples = [ " ".join(word_tokenize(sample.strip())) for sample in samples if sample.strip() != "" ]

# Predict
predictions, raw_outputs = ner_model.predict(samples)

In [ ]:
# Group the Predicted Entities
entities = []
for pred in predictions:
  grouped_entities = []
  for rec in pred:

    token = list(rec.keys())[0]
    label = rec[token]

    if label == "O":
      continue

    if "B-" in label:
      grouped_entities.append({"token": token, "label": label.replace("B-","")})
    elif "I-" in label and len(grouped_entities) > 0:
      grouped_entities[-1]["token"] += f" {token}"

  entities.append(grouped_entities)

# Print the model outputs
for sample, results in zip(samples, entities):
  print(sample)
  for res in results:
    print("\t", res["token"], "=>", res["label"])
  print("==================")

تلقى تعليمه في الكتاب ثم انضم الى الأزهر عام 1873م . تعلم على يد السيد جمال الدين الأفغاني والشيخ محمد عبده
	 الأزهر => organization
	 عام 1873م => time
	 جمال الدين الأفغاني => person
	 محمد عبده => person
بعد عودته إلى القاهرة، التحق نجيب الريحاني فرقة جورج أبيض، الذي كان قد ضمَّ - قُبيل ذلك - فرقته إلى فرقة سلامة حجازي . و منها ذاع صيته
	 القاهرة، => location
	 نجيب الريحاني => person
	 فرقة جورج أبيض، => organization
	 فرقة سلامة حجازي => organization
امبارح اتفرجت على مباراة مانشستر يونايتد مع ريال مدريد في غياب الدون كرستيانو رونالدو
	 مانشستر يونايتد => organization
	 ريال مدريد => organization
	 كرستيانو رونالدو => person
Government extends flight ban from India , Pakistan until June 21
	 India => location
	 Pakistan => location
	 June 21 => time
